### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 111.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.71
    Uninstalling botocore-1.29.71:
      Successfully uninstalled botocore-1.29.71
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.71
    Uninstalling boto3-1.26.71:
      Successfully uninstalled boto3-1.26.71
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.71 requires botocore==1.29.71, but you have botocore 1.29.116 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.3/743.3 kB 14.5 MB/s

In [6]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


### chatglm lora方式（simple_thu_chatglm6b单机单卡）
1: 使用羊驼语料数据  
2：语料处理，tokenization及label标注  
3：HF trainer API  
 

In [ ]:
!git clone https://github.com/hikariming/alpaca_chinese_dataset.git
!git clone https://github.com/yuanzhoulvpi2017/zero_nlp.git

In [ ]:
from glob import glob
import os 
import pandas as pd 
import shutil
from itertools import chain
from tqdm import tqdm
from pathlib import Path
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
target_dir_list = ['alpaca_chinese_dataset/其他中文问题补充/',
                   'alpaca_chinese_dataset/翻译后的中文数据/',
                   'alpaca_chinese_dataset/chatglm问题数据补充/',
                #    'alpaca_chinese_dataset/原始英文数据/'
                   ]

all_json_path = [glob(i+"*.json") for i in target_dir_list]
all_json_path = list(chain(*all_json_path))
len(all_json_path), all_json_path[:5]

In [ ]:
def read_json(x:str):
    try:
        data = pd.read_json(x)
        return data 
    except Exception as e:
        return pd.DataFrame()

alldata = pd.concat([read_json(i) for i in all_json_path])
# alldata

In [ ]:
genrate_data_dir = "data3_0328"
genrate_data_dir = Path(genrate_data_dir)

if genrate_data_dir.exists():
    shutil.rmtree(genrate_data_dir, ignore_errors=True)

os.makedirs(genrate_data_dir, exist_ok=True)

In [ ]:
alldata = alldata.sample(frac=1).reset_index(drop=True)

chunk_size = 666

for index, start_id in tqdm(enumerate(range(0, alldata.shape[0], chunk_size))):
    temp_data = alldata.iloc[start_id:(start_id+chunk_size)]
    temp_data.to_csv(genrate_data_dir.joinpath(f"{index}.csv"), index=False)

In [ ]:
# from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
# from thuglmcode.model_chatglm import ChatGLMForConditionalGeneration
from transformers import Trainer, TrainingArguments
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from peft import get_peft_model, LoraConfig, TaskType
from typing import Optional
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("yuanzhoulvpi/chatglm6b-dddd", trust_remote_code=True)

model = AutoModel.from_pretrained(
    "yuanzhoulvpi/chatglm6b-dddd", trust_remote_code=True).half().cuda()

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    # ['dense','dense_h_to_4h','dense_4h_to_h'] # 'query_key_value',
    target_modules=['query_key_value',],
)
model = get_peft_model(model, peft_config)

class MyTrainer(Trainer):
    def _save(self, output_dir: Optional[str] = None, state_dict=None):
        # If we are executing this function, we are the process zero, so we don't check for that.
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        os.makedirs(output_dir, exist_ok=True)
        def save_tunable_parameters(model, path):
            saved_params = {
                k: v.to("cpu") for k, v in model.named_parameters() if v.requires_grad
            }
            # saved_params = model.state_dict()
            torch.save(saved_params, path)

        save_tunable_parameters(
            self.model, os.path.join(output_dir, "chatglm-lora.pt")
        )

In [ ]:
random.seed(42)

all_file_list = glob(pathname=genrate_data_dir.joinpath("*.csv").__str__())

test_file_list = random.sample(all_file_list, int(len(all_file_list)*0.25))
train_file_list = [i for i in all_file_list if i not in test_file_list]

len(train_file_list), len(test_file_list)

dataset = load_dataset(
    "csv",
    data_files={
    'train':train_file_list,
    'valid':test_file_list
    },
    cache_dir="cache_data"
)
dataset

In [ ]:
def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids

def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids) + 1
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [tokenizer.eop_token_id]
            + [-100] * (longest - ids_l - 1)
        )
        ids = ids + [tokenizer.eop_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }

In [ ]:
def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    # {"context": context, "target": target}
    example['context'] = context
    example['target'] = target
    return example

max_seq_length = 512

def preprocess(example):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target, max_length=max_seq_length, truncation=True, add_special_tokens=False
    )
    input_ids = prompt_ids + target_ids + [tokenizer.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}

def filter_nan(example):
    return example['target'] is not None and example['context'] is not  None


tokenized_datasets = dataset.map(
    function=format_example, remove_columns=dataset['train'].column_names
    ).filter(function=filter_nan)
tokenized_datasets = tokenized_datasets.map(function=preprocess)
tokenized_datasets

In [ ]:
from transformers.trainer_callback import TrainerCallback, TrainerState, TrainerControl



class EmptyCacheCallBack(TrainerCallback):
    """
    通过callback的形式，解决显存不够的问题

    """

    def __init__(self) -> None:
        super().__init__()

    def on_log(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, logs, **kwargs):
        """
        Event called after logging the last logs.
        """
        torch.cuda.empty_cache()

    def on_epoch_begin(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        torch.cuda.empty_cache()

    def on_step_begin(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        torch.cuda.empty_cache()
        
    

eccb = EmptyCacheCallBack()

In [ ]:
args = TrainingArguments(
    output_dir="test004",
    per_device_train_batch_size=2, 
    per_device_eval_batch_size=1,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=100,
    fp16=True,
    push_to_hub=False,
    remove_unused_columns=False
)

trainer = MyTrainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    # callbacks=[eccb]
)
trainer.train()

### chatglm lora方式（单机多卡、模型并行）
与单机单卡训练类似，使用pytorch ddp+lora

In [ ]:
#print('s3://{}/llm/models/'.format(sagemaker_session.default_bucket()))
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ChatGLM-6B/lora_tuning/

In [2]:
!rm -rf ChatGLM-6B/lora_tuning
!mkdir -p ChatGLM-6B/lora_tuning && cd ChatGLM-6B/lora_tuning/ && git clone https://github.com/qingyuan18/zero_nlp.git
!cd ChatGLM-6B/lora_tuning/zero_nlp/Chatglm6b_ModelParallel/ && ls -lt

Cloning into 'zero_nlp'...
remote: Enumerating objects: 569, done.
remote: Counting objects: 100% (268/268), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 569 (delta 174), reused 191 (delta 100), pack-reused 301
Receiving objects: 100% (569/569), 29.07 MiB | 23.09 MiB/s, done.
Resolving deltas: 100% (316/316), done.
total 208
drwxrwxr-x 2 ec2-user ec2-user   4096 May  3 08:23 images
-rw-rw-r-- 1 ec2-user ec2-user   3681 May  3 08:23 readme.md
drwxrwxr-x 2 ec2-user ec2-user   4096 May  3 08:23 thuglm
-rw-rw-r-- 1 ec2-user ec2-user  11564 May  3 08:23 train_model_all.py
-rw-rw-r-- 1 ec2-user ec2-user 187995 May  3 08:23 MyTrainer.py


In [ ]:
!git clone https://github.com/hikariming/alpaca_chinese_dataset.git
!rm -rf alpaca_chinese_dataset/原始英文数据
!ChatGLM-6B/lora_tuning/s5cmd sync alpaca_chinese_dataset/ s3://{bucket}/llm/chatglm/alpaca_chinese_datasets/

In [17]:
import json
import os
instance_type = 'ml.p4d.24xlarge'
if instance_type in [
    "ml.p3.16xlarge",
    "ml.p3dn.24xlarge",
    "ml.g5.48xlarge",
    "ml.p4d.24xlarge",
]:
    processes_per_host = 8
elif instance_type == "ml.p2.16xlarge":
    processes_per_host = 16
else:
    processes_per_host = 4

print("processes_per_host is set to:", processes_per_host)

####CUDA visible divices ##############
CUDA_VISIBLE_DEVICES = ""
for i in range(processes_per_host):
    CUDA_VISIBLE_DEVICES = CUDA_VISIBLE_DEVICES+ str(i) + ","
os.environ["CUDA_VISIBLE_DEVICES"] = CUDA_VISIBLE_DEVICES[:-1]
#print(os.environ["CUDA_VISIBLE_DEVICES"])

#####device mapping ########################
DEVICE_MAP_DICT = {"transformer.word_embeddings": 0,
                   "transformer.layers.0": 0,
                   "transformer.layers.1": 0,
                   "transformer.layers.2": 0,
                   "transformer.layers.3": 0,
                   "transformer.layers.4": 0,
                   "transformer.layers.5": 0,
                   "transformer.layers.6": 0,
                   "transformer.layers.7": 1,
                   "transformer.layers.8": 1,
                   "transformer.layers.9": 1,
                   "transformer.layers.10": 1,
                   "transformer.layers.11": 1,
                   "transformer.layers.12": 1,
                   "transformer.layers.13": 1,
                   "transformer.layers.14": 2,
                   "transformer.layers.15": 2,
                   "transformer.layers.16": 2,
                   "transformer.layers.17": 2,
                   "transformer.layers.18": 2,
                   "transformer.layers.19": 2,
                   "transformer.layers.20": 2,
                   "transformer.layers.21": 3,
                   "transformer.layers.22": 3,
                   "transformer.layers.23": 3,
                   "transformer.layers.24": 3,
                   "transformer.layers.25": 3,
                   "transformer.layers.26": 3,
                   "transformer.layers.27": 3,
                   "transformer.final_layernorm": 1,
                   "lm_head": 1
                   }

os.environ['DEVICE_MAP_DICT']=json.dumps(DEVICE_MAP_DICT)
device_map_dict = json.loads(os.environ['DEVICE_MAP_DICT'])
#print(device_map_dict)
#print((os.environ['DEVICE_MAP_DICT']))

processes_per_host is set to: 8
{'transformer.word_embeddings': 0, 'transformer.layers.0': 0, 'transformer.layers.1': 0, 'transformer.layers.2': 0, 'transformer.layers.3': 0, 'transformer.layers.4': 0, 'transformer.layers.5': 0, 'transformer.layers.6': 0, 'transformer.layers.7': 1, 'transformer.layers.8': 1, 'transformer.layers.9': 1, 'transformer.layers.10': 1, 'transformer.layers.11': 1, 'transformer.layers.12': 1, 'transformer.layers.13': 1, 'transformer.layers.14': 2, 'transformer.layers.15': 2, 'transformer.layers.16': 2, 'transformer.layers.17': 2, 'transformer.layers.18': 2, 'transformer.layers.19': 2, 'transformer.layers.20': 2, 'transformer.layers.21': 3, 'transformer.layers.22': 3, 'transformer.layers.23': 3, 'transformer.layers.24': 3, 'transformer.layers.25': 3, 'transformer.layers.26': 3, 'transformer.layers.27': 3, 'transformer.final_layernorm': 1, 'lm_head': 1}


In [ ]:
# define Training Job Name 
job_name = f'huggingface-chatglm-loRA-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/loRA/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/alpaca-chinese-dataset-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'



instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_s3_path,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : processes_per_host，
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'DEVICE_MAP_DICT'        : DEVICE_MAP_DICT,
              'CUDA_VISIBLE_DEVICES'   : CUDA_VISIBLE_DEVICES,
              'DEVICE_MAP_DICT' : json.dumps(DEVICE_MAP_DICT),
              'PER_DEVICE_TRAIN_BATCH_SIZE' : 1,
              'PER_DEVICE_EVAL_BATCH_SIZE': 8,
              'GRADIENT_ACCMULATION_STEPS': 8,
              'NUM_TRAIN_EPOCHS'       : 8,
              'SAVE_STEPS'             :20
    
}

inputs={
   'alpaca-chinese-dataset': f"s3://{bucket}/llm/chatglm/alpaca_chinese_datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'train_model_all.py',          # user endpoint script
    source_dir           = 'ChatGLM-6B/lora_tuning/zero_nlp/Chatglm6b_ModelParallel',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

In [ ]:
# # 完整的训练流程
# 1. 数据基于`https://github.com/hikariming/alpaca_chinese_dataset`
# 2. 部分代码来源于`https://github.com/27182812/ChatGLM-chinese-insturct/blob/main/finetune.py`
# 3. 基于我之前修改的`model_chatglm.py`做的一整套教程
# 
# ## 清洗数据

# 在这里控制要使用的显卡
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
# 如果没有下载这个仓库，可以使用下面命令进行clone

# !git clone https://github.com/hikariming/alpaca_chinese_dataset.git


########################################################################################################################
# part 1 数据准备
########################################################################################################################


from glob import glob
import os
import pandas as pd
import shutil
from itertools import chain
from tqdm import tqdm
from pathlib import Path

target_dir_list = ['alpaca_chinese_dataset/其他中文问题补充/',
                   'alpaca_chinese_dataset/翻译后的中文数据/',
                   'alpaca_chinese_dataset/chatglm问题数据补充/',
                   #    'alpaca_chinese_dataset/原始英文数据/'
                   ]

all_json_path = [glob(i + "*.json") for i in target_dir_list]
all_json_path = list(chain(*all_json_path))
len(all_json_path), all_json_path[:5]


def read_json(x: str):
    try:
        data = pd.read_json(x)
        return data
    except Exception as e:
        return pd.DataFrame()


alldata = pd.concat([read_json(i) for i in all_json_path])

genrate_data_dir = "data3_0328"
genrate_data_dir = Path(genrate_data_dir)

if genrate_data_dir.exists():
    shutil.rmtree(genrate_data_dir, ignore_errors=True)

os.makedirs(genrate_data_dir, exist_ok=True)

alldata = alldata.sample(frac=1).reset_index(drop=True)

chunk_size = 666

for index, start_id in tqdm(enumerate(range(0, alldata.shape[0], chunk_size))):
    temp_data = alldata.iloc[start_id:(start_id + chunk_size)]
    temp_data.to_csv(genrate_data_dir.joinpath(f"{index}.csv"), index=False)



########################################################################################################################
# part 2 模型加载和转换
########################################################################################################################



from thuglm.modeling_chatglm import ChatGLMForConditionalGeneration
from transformers import Trainer
from transformers import TrainingArguments
import random
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from peft import get_peft_model, LoraConfig, TaskType
from typing import Optional
import torch
from MyTrainer import Trainer

tokenizer = AutoTokenizer.from_pretrained("thuglm", trust_remote_code=True)

device_map_dict = {'transformer.word_embeddings': 0,
                   'transformer.layers.0': 0,
                   'transformer.layers.1': 0,
                   'transformer.layers.2': 0,
                   'transformer.layers.3': 0,
                   'transformer.layers.4': 0,
                   'transformer.layers.5': 0,
                   'transformer.layers.6': 0,
                   'transformer.layers.7': 0,
                   'transformer.layers.8': 0,
                   'transformer.layers.9': 0,
                   'transformer.layers.10': 0,
                   'transformer.layers.11': 0,
                   'transformer.layers.12': 0,
                   'transformer.layers.13': 0,
                   'transformer.layers.14': 0,
                   'transformer.layers.15': 1,
                   'transformer.layers.16': 1,
                   'transformer.layers.17': 1,
                   'transformer.layers.18': 1,
                   'transformer.layers.19': 1,
                   'transformer.layers.20': 1,
                   'transformer.layers.21': 1,
                   'transformer.layers.22': 1,
                   'transformer.layers.23': 1,
                   'transformer.layers.24': 1,
                   'transformer.layers.25': 1,
                   'transformer.layers.26': 1,
                   'transformer.layers.27': 1,
                   'transformer.final_layernorm': 1,
                   'lm_head': 1
                   }

model = AutoModel.from_pretrained(
    "thuglm", trust_remote_code=True).half().cuda()

for k, v in device_map_dict.items():
    if k == 'transformer.word_embeddings':
        model.transformer.word_embeddings = model.transformer.word_embeddings.to(f'cuda:{v}')
    if k.find("transformer.layers") != -1:
        sub_value = int(k.replace("transformer.layers.", ""))
        model.transformer.layers[sub_value] = model.transformer.layers[sub_value].to(f'cuda:{v}')

    if k == "transformer.final_layernorm":
        model.transformer.final_layernorm = model.transformer.final_layernorm.to(f'cuda:{v}')

model.enable_input_require_grads()
torch.cuda.empty_cache()

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,
    # ['dense','dense_h_to_4h','dense_4h_to_h'] # 'query_key_value',
    target_modules=['query_key_value', ],
)
model = get_peft_model(model, peft_config)

for k, v in device_map_dict.items():
    #     if k == 'transformer.word_embeddings':
    #         model.base_model.transformer.word_embeddings = model.base_model.transformer.word_embeddings.to(f'cuda:{v}')
    if k.find("transformer.layers") != -1:
        sub_value = int(k.replace("transformer.layers.", ""))
        model.base_model.transformer.layers[sub_value].attention.query_key_value.lora_A = \
            model.base_model.transformer.layers[sub_value].attention.query_key_value.lora_A.to(f'cuda:{v}')
        model.base_model.transformer.layers[sub_value].attention.query_key_value.lora_B = \
            model.base_model.transformer.layers[sub_value].attention.query_key_value.lora_B.to(f'cuda:{v}')

########################################################################################################################
# part 3 数据加载
########################################################################################################################


random.seed(42)

all_file_list = glob(pathname=genrate_data_dir.joinpath("*.csv").__str__())

test_file_list = random.sample(all_file_list, int(len(all_file_list) * 0.25))
train_file_list = [i for i in all_file_list if i not in test_file_list]
train_file_list, test_file_list = train_file_list[:5], test_file_list[:5]

len(train_file_list), len(test_file_list)

dataset = load_dataset(
    "csv",
    data_files={
        'train': train_file_list,
        'valid': test_file_list
    },
    cache_dir="cache_data"
)


def get_masks_and_position_ids(
        seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
            seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1:] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids) + 1
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
                [-100] * (seq_len - 1)
                + ids[(seq_len - 1):]
                + [tokenizer.eos_token_id]
                + [-100] * (longest - ids_l - 1)
        )
        ids = ids + [tokenizer.eos_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }


def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    # {"context": context, "target": target}
    example['context'] = context
    example['target'] = target
    return example


max_seq_length = 1024


def preprocess(example):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True)
    target_ids = tokenizer.encode(
        target, max_length=max_seq_length, truncation=True, add_special_tokens=False
    )
    input_ids = prompt_ids + target_ids + [tokenizer.eos_token_id]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}


def filter_nan(example):
    return example['target'] is not None and example['context'] is not None


tokenized_datasets = dataset.map(
    function=format_example, remove_columns=dataset['train'].column_names
).filter(function=filter_nan)
tokenized_datasets = tokenized_datasets.map(function=preprocess)


########################################################################################################################
# part 4 训练
########################################################################################################################


args = TrainingArguments(
    output_dir="modellog0040101",
    per_device_train_batch_size=4,  # 如果在24G显存上的显卡，可以开到4
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=20,
    fp16=True,
    push_to_hub=False,
    remove_unused_columns=False
)

model.is_parallelizable = True
model.model_parallel = True

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)
trainer.train()
